This notebook is a companion to the [Query Atlas with Natural Language Using LangChain and LangGraph](https://www.mongodb.com/docs/atlas/atlas-vector-search/ai-integrations/langchain/natural-language-to-mql/) tutorial. Refer to the page for set-up instructions and detailed explanations.

This notebook demonstrates how to query an Atlas cluster with a natural language prompt using an AI agent built with the [LangChain MongoDB Toolkit](https://langchain-mongodb.readthedocs.io/en/latest/langchain_mongodb/agent_toolkit/langchain_mongodb.agent_toolkit.toolkit.MongoDBDatabaseToolkit.html#langchain_mongodb.agent_toolkit.toolkit.MongoDBDatabaseToolkit) and the [LangGraph ReAct Agent Framework](https://langchain-ai.github.io/langgraph/agents/agents/).

<a target="_blank" href="https://colab.research.google.com/github/mongodb/docs-notebooks/blob/main/ai-integrations/langchain-natural-language.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [ ]:
pip install --quiet --upgrade langchain-mongodb langchain-openai langgraph

## Set up your environment

Before you begin, make sure you have the following:

- An Atlas cluster up and running (you'll need the [connection string](https://www.mongodb.com/docs/guides/atlas/connection-string/))
- An API key to access an LLM (This tutorial uses a model from OpenAI, but you can use any model [supported by LangChain](https://python.langchain.com/docs/integrations/chat/))

In [ ]:
os.environ["OPENAI_API_KEY"] = '<api-key>'
ATLAS_CONNECTION_STRING = '<atlas-connection-string>'
ATLAS_DB_NAME = 'sample_restaurants'
NATURAL_LANGUAGE_QUERY = 'Find all restaurants that serve hamburgers.'

In [ ]:
import os
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent
from langchain_mongodb.agent_toolkit import (
    MONGODB_AGENT_SYSTEM_PROMPT,
    MongoDBDatabase,
    MongoDBDatabaseToolkit,
)

## Build the agent

Next, define the `NaturalLanguageToMQL` Python class.

In [ ]:
class NaturalLanguageToMQL:
    def __init__(self):
        self.llm = ChatOpenAI(model="gpt-4o-mini", timeout=60)
        self.system_message = MONGODB_AGENT_SYSTEM_PROMPT.format(top_k=5)
        self.db_wrapper = MongoDBDatabase.from_connection_string(
                            ATLAS_CONNECTION_STRING, 
                            database=ATLAS_DB_NAME)
        self.toolkit = MongoDBDatabaseToolkit(db=self.db_wrapper, llm=self.llm)
        self.agent = create_react_agent(
                        self.llm, 
                        self.toolkit.get_tools(), 
                        state_modifier=self.system_message)
        self.messages = []

    def convert_to_mql_and_execute_query(self, query):
        # Start the agent with the agent.stream() method
        events = self.agent.stream(
            {'messages': [('user', query)]},
            stream_mode='values',
        )
        # Add output (events) from the agent to the self.messages list
        for event in events:
            self.messages.extend(event['messages'])
    
    def print_results(self):
        # Print the the end-user's expected output from 
        # the final message produced by the agent.
        print(self.messages[-1].content)

## Run a sample query

And finally, instantiate the `NaturalLanguageToMQL` class and run a sample query.

In [ ]:
def main():
    converter = NaturalLanguageToMQL()
    converter.convert_to_mql_and_execute_query(NATURAL_LANGUAGE_QUERY)
    converter.print_results()

main()